<a href="https://colab.research.google.com/github/egonik-unlp/random_projects/blob/master/data_analytics/dataset_kaggle_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Procesamiento del Dataset Hazardous Air Pollutants {display-mode: "form"}

# This code will be hidden when the notebook is loaded.
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Carga de datos desde Kaggle

Primero para poder trabajar, cargamos nuestra clave de API de kaggle y activamos la libreria

In [ ]:
from google.colab import files
!wget https://raw.githubusercontent.com/egonik-unlp/random_projects/master/data_analytics/kaggle.json
!mkdir /root/.kaggle 
!cp kaggle.json /root/.kaggle
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


Luego descargamos el dataset y lo descomprimimos en la carpeta datasets

In [ ]:
!kaggle datasets download -d 'epa/hazardous-air-pollutants'
!unzip /content/hazardous-air-pollutants.zip -d /content/dataset

hazardous-air-pollutants.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/hazardous-air-pollutants.zip
replace /content/dataset/epa_hap_daily_summary.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Importamos las librerias que nos serán útiles para procesar los datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

Cargamos el dataset como un DataFrame de pandas

In [ ]:
# Es muy pesado de cargar, lo pongo en una celda individual para asegurarme de que no se corra dos veces
df = pd.read_csv('dataset/epa_hap_daily_summary.csv')

Renombramos columnas

In [ ]:
df.rename( columns={
    'state_code': 'state_id',
    'county_code':'county_id',
    'parameter_code':'parameter_id',
    'method_code':'method_id',
    'site_num': 'site_id',
    'date_local':'sample_date',
}, inplace = True)

In [ ]:
df['sample_id'] = df.index

In [ ]:
#preparacion para generar datos unicos

x,y = df[['state_id','county_id' ]].astype(str).iteritems()
df['county_unique_id'] = x[1] + y[1]
x,y = df[['county_unique_id','site_id' ]].astype(str).iteritems()
df['site_unique_id'] = x[1] + y[1]

In [ ]:
def remove_n_rows(N:int, dataframe:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
  '''
  N representa la fraccion en la que quiero reducir el dataframe
  '''

  number_of_rows = len(dataframe) -  int(len(dataframe)/N)
  drop_indices = np.random.choice(dataframe.index ,number_of_rows,replace = False)
  print(f'El dataframe original tenia {len(dataframe)} rows, se redujeron a {len(dataframe) - len(drop_indices)}' )
  return dataframe.drop(drop_indices)

df2 = remove_n_rows(15.01,df)

El dataframe original tenia 8097069 rows, se redujeron a 539444


In [ ]:
state = df2[[
            'state_name',
            'state_id'
]].copy()

county= df2[[
          'county_unique_id',
          'county_name',
          'state_id'
]].copy()

site = df2[[
         'address',
         'city_name',
         'cbsa_name',
         'county_unique_id',
         'site_unique_id'
]].copy()

sample = df2[[
           'sample_id',
           'sample_duration',
           'sample_date',
           'units_of_measure',
           'event_type',
           'observation_count',
           'arithmetic_mean',
           'site_unique_id',
           'parameter_id',
           'method_id'
]].copy()

parameter = df2[[
              'parameter_id',
              'parameter_name'
]].copy()

method = df2[[
           'method_id',
           'method_name'
]].copy()

In [ ]:
method.drop_duplicates('method_id', inplace = True)
state.drop_duplicates('state_id', inplace = True)
sample.drop_duplicates('sample_id', inplace = True)
parameter.drop_duplicates('parameter_id', inplace = True)
county.drop_duplicates('county_unique_id', inplace = True)
site.drop_duplicates('site_unique_id', inplace = True)
state['alpha_code'] = state['state_name'].map(us_state_abbrev)
state.replace(np.nan, '-', inplace = True)

In [ ]:
def save_xls(list_dfs, xls_path, list_names):
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,list_names[n], index = False)
        writer.save()
dfs = [state, county,site, sample, parameter, method]
dfs_names = ['state', 'county','site', 'sample', 'parameter', 'method']






save_xls(dfs, f'database_{len(df2)}.xlsx', dfs_names)